In [1]:
# Imports
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Read Raw data
df = pd.read_csv("DRc5a2011-2014.txt",sep=",")

In [3]:
# Read IDC code groups description
idc_groups = pd.read_csv("idc_codes.txt",sep=";")

In [4]:
# Initial filtering
# Reg == 1100 - Whole Russia
# CCl == 10 - IDC 10 codes
# Group == T - urban and rural population together
df = df[(df.Reg == 1100) & (df.CCl == 10) & (df.Group == "T")].drop(['Reg','Group','CCl'], 1)

## Prepare data for gender/age plot

In [54]:
# Remove info about cause of death
by_gender_and_age = df[(df.Cause == 0)].drop(['Cause'], 1)

In [55]:
# Convert group names to integers
new_columns = []
for column in by_gender_and_age.columns:
    if column[:4] == 'Drac':
        new_columns.append(int(column[4:]))
    else:
        new_columns.append(column)
        
by_gender_and_age.columns = new_columns

In [56]:
gae_columns = ['Year', 'Sex',  'Age Group', 'Deaths']
gae = pd.DataFrame([], columns=gae_columns)
for index, row in by_gender_and_age.iterrows():
    sex = "Male" if (row['Sex'] == "M") else "Female"
    for i in [0, 1] + range(5, 90, 5):
        gae = gae.append(pd.DataFrame([[row['Year'], sex, i, row[i]]], columns=gae_columns))

In [57]:
# Export to JSON file
gae.to_json('../data/by_gender_and_age.json', 'records')